# Alpha-Fold Metainference for structural ensemble prediction of a partially disordered protein.

In [ ]:
!pwd

### Only run the following cell if you are in Google Colab

In [ ]:
#Install collab
!pip install -q condacolab
import condacolab
condacolab.install()

### Install dependencies 

In [ ]:
import os
home=os.getcwd()
os.chdir(home)

#Download OpenMM-Plumed-MPI patch
!conda install -y -c conda-forge mpich mpi4py openmm=8.0 plumed=2.8.2=mpi_mpich_h7ded119_0 py-plumed cmake swig pandas mdtraj biopython matplotlib gromacs
!conda install -y -c anaconda ipykernel
!conda install -y -c bioconda pulchra

!git clone https://github.com/vendruscolo-lab/OpenMM-Plumed-MPI

os.chdir(home+'/OpenMM-Plumed-MPI')

#Build openmm-plumed-mpi
!mkdir build install openmm -p plumed/include -p plumed/lib
!unzip openmm.zip -d openmm
!unzip plumed_lib.zip -d plumed/lib
!unzip plumed_include.zip -d plumed/include
os.chdir(os.getcwd()+'/build')
!cmake ..
!make
!make install
!make PythonInstall
os.chdir(os.getcwd()+'/../install/lib')
!cp -r * /usr/local/lib/

In [ ]:
!pwd

# Run Alpha-Fold distance map prediction


__For this example (TDP-43 WtoA):__

The AF distance map has already been calculated and it is loaded in this notebook below.

__For arbitrary protein sequences:__

*   Open [this](https://github.com/zshengyu14/ColabFold_distmats/blob/main/AlphaFold2.ipynb) link and chose colab.
*   Input the protein sequence  as query sequence.
*   The rest of the options remain default and cells are run until the end.
*   Download the link with the AF data and upload it as AF_data in AlphaFold-IDP folder

# Setup protein system in CALVADOS and OPENMM

In [ ]:
os.chdir(home)
!git clone https://github.com/vendruscolo-lab/AlphaFold-IDP

In [ ]:
os.chdir(home+'/AlphaFold-IDP/prep_run')

In [ ]:
import shutil
import csv
dir=os.getcwd()
###################### The entries below need to be adapted in eac simulation ######################
#TDP-43 sequence
fasta_sequence="MSEYIRVTEDENDEPIEIPSEDDGTVLLSTVTAQFPGACGLRYRNPVSQCMRGVRLVEGILHAPDAGAGNLVYVVNYPKDNKRKMDETDASSAVKVKRAVQKTSDLIVLGLPAKTTEQDLKEYFSTFGEVLMVQVKKDLKTGHSKGFGFVRFTEYETQVKVMSQRHMIDGRACDCKLPNSKQSQDEPLRSRKVFVGRCTEDMTEDELREFFSQYGDVMDVFIPKPFRAFAFVTFADDQIAQSLCGEDLIIKGISVHISNAEPKHNSNRQLERSGRFGGNPGGFGNQGGFGNSRGGGAGLGNNQGSNMGGGMNFGAFSINPAMMAAAQAALQSSAGMMGMLASQQNQSGPSGNNQNQGNMQREPNQAFGSGNNSYSGSNSGAAIGAGSASNAGSGSGFNGGFGSSMDSKSSGAGM"
#Conditions
pH=7.4
temp=298
ionic=0.2
PAE_cut=5
Pr_cut=0.2
NR=2
#Decide the plddt based ordered (od) and disordered (dd) regions
ordered_domains = {'od1': [3, 79], 'od2': [104,178],'od3':[191,260]}
disordered_domains={'dd1': [1,2],'dd2':[80,103],'dd3':[179,190],'dd4':[261,414]}

#AF input created in AF-distance map prediction and used in AF-MI
pdb_af='tdp43_WtoA_bf4cc_unrelaxed_rank_001_alphafold2_ptm_model_3_seed_000.pdb'
json_af='tdp43_WtoA_bf4cc_predicted_aligned_error_v1.json'
npy_af='alphafold2_ptm_model_3_seed_000_prob_distributions.npy'
mean_af='alphafold2_ptm_model_3_seed_000_mean.csv'
#Copy AF data
shutil.copy2(dir+"/../AF_DATA/"+pdb_af, dir+"/pdb_af.pdb")
shutil.copy2(dir+"/../AF_DATA/"+json_af, dir+"/pae.json")
shutil.copy2(dir+"/../AF_DATA/tdp43_WtoA_bf4cc_distmat/"+mean_af, dir+"/mean_af.csv")
shutil.copy2(dir+"/../AF_DATA/tdp43_WtoA_bf4cc_distmat/"+npy_af, dir+"/prob.npy")
####################################################################################################

f = open("sequence.dat", "w")
f.write(fasta_sequence)
f.close()
#Write the csv files
with open('ordered_domains.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, ordered_domains.keys())
    w.writeheader()
    w.writerow(ordered_domains)
with open('disordered_domains.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, disordered_domains.keys())
    w.writeheader()
    w.writerow(disordered_domains)

#Copy OPENMM calvados forcefield files
shutil.copy2(dir+"/../scripts_prep/gen_xml_and_constraints.py", dir)
shutil.copy2(dir+"/../scripts_prep/residues.csv", dir)

path_gen_xml = dir+'/gen_xml_and_constraints.py sequence.dat '+str(pH)+' '+str(temp)+' '+str(ionic)
print(path_gen_xml)
os.system(f'python {path_gen_xml}')



In [ ]:
#Make plumed files.
#Copy and run the prep script that makes the plumed file.
#The Collective variables (CVs) in these case are chosen to be the torsion angles between structured domains.
import subprocess
shutil.copy2(dir+"/../scripts_prep/make_plumed_distmat.py", dir)
subprocess.run(['python', str(dir)+'/make_plumed_distmat.py', 'sequence.dat',str(PAE_cut), str(Pr_cut)], capture_output=True, text=True)

In [ ]:
!cat plumed.dat

In [ ]:
#Make the plumed_analysis.dat file
shutil.copy2(dir+"/../scripts_prep/make_plumed_analysis.py", dir)
path_gen_analysis = dir+'/make_plumed_analysis.py sequence.dat'
os.system(f'python {path_gen_analysis}')



__Note:__ that users needs to fill in the respective PLUMED (_ _FILL _ _) PBMETAD and Metainference parameters according to the problem in need.

__Note:__ This tutorial assumes that you know [Parallel Bias Metadynamics](https://www.plumed.org/doc-v2.9/user-doc/html/_p_b_m_e_t_a_d.html) and [Metainference](https://link.springer.com/content/pdf/10.1007/978-1-4939-9608-7_13.pdf) theory and practice.


### In case you are running the TDP-43 WtoA example:

You can directly use the PBMetaD and Metainference parameters just as copied below by executing the next shell. Otherwise you need to define your system specific parameters.

In [ ]:
shutil.copy2(dir+"/../scripts_prep/plumed_TDP-43.dat", dir+'/plumed.dat')
shutil.copy2(dir+"/../scripts_prep/plumed_analysis_TDP-43.dat", dir+'/plumed_analysis.dat')


# Energy minimization

In [ ]:
#Activate the conda openmm-plumed environment
shutil.copy2(dir+"/../scripts_prep/simulate_em.py", dir)
#First run a short minimization
simulate_em = dir+'/simulate_em.py '+str(pH)+' '+str(temp)

os.system(f'python {simulate_em}')



# Run AF-MI

In [ ]:
#and run AF-MI
shutil.copy2(dir+"/../scripts_prep/simulate.py", dir)
print(dir)
#simulate= dir+'/simulate.py '+str(pH)+' '+str(temp)
#os.system(f'mpirun -np 6 python  {simulate}')
##param1 = "value1" param2 = "value2" !python script.py {param1} {param2}

!mpirun -np {NR} python simulate.py {pH} {temp}

In [ ]:
!tail COLVAR.1

# Analysis

In [ ]:
!cp ../scripts_prep/* .

!zip files.zip script.sh  pulchra.sh pulchra.py  backmap.py simulate*.py  dcd2xtc.py plumed_analysis.dat reconstruct.dat  resample.py  fes2.py  sequence.dat plumed.dat struct*pdb input_af.pdb r1_excl.pkl forcefield.xml residues.csv *npy *mean*csv pdb_af.pdb  keepH.sh

!mkdir analysis
!cp files.zip analysis/
os.chdir(dir+"/analysis")
!unzip files.zip

Generate the atomistic structural ensemble

In [ ]:
!pwd
!chmod 755 script.sh
!./script.sh {NR}
#The final atomistic structural ensemble.
!ls segment_5_input_af_rebuilt.xtc

Generate the FES along collective variables

In [ ]:
#Time depentent FES
#For other proteins the entries CV1,CV2,CV3 etc need to follow the COLVAR columns like:
#for i in $(echo CV1 CV2 CV3 etc);do
!num=1
## For TDP-43 WtoA
!for i in $(echo Rg Rg1 Rg2 Rg3 Rg4 torsion1 torsion2 RMSD1 RMSD2 RMSD3);do python fes2.py --CV_col $num --CV_name $i ; num=$((num+1)) ; echo $num; done

Root mean square fluctuations per residue

In [ ]:
!echo "0" |gmx rmsf -f segment_5_input_af_rebuilt.xtc -s  segment_5_input_af_0_sys.pdb -res -o rmsf.xvg
import numpy as np
from matplotlib import  pyplot as plt
x=np.loadtxt("rmsf.xvg",comments=['#', '$', '@'])[:, 0]
y=np.loadtxt("rmsf.xvg",comments=['#', '$', '@'])[:, 1]
#print(x)
#print(y)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(x,y,color="black",alpha=0.5)

plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel('Residue',fontsize=15)
plt.ylabel('RMSF (nm)',fontsize=15)
ax.tick_params(axis='both', labelsize=15)
ax.legend=None
plt.savefig('rmsf.pdf',bbox_inches='tight')

Download files

In [ ]:
print(home)
!pwd
!rm segment_5_input_af_*.rebuilt.xtc segment_5_input_af_*.rebuilt.pdb
!zip -r {home}/archive.zip {home}/AlphaFold-IDP/prep_run
#!zip -r {home}/archive.zip *png ../plumed.dat plumed_analysis.dat reconstruct.dat segment_5_input_af_rebuilt.xtc segment_5_input_af_0_sys.pdb rmsf.pdf ../pae_m.png FES*png FULLBIAS COLVAR ../HILLS* ../COLVAR*

In [ ]:
from google.colab import files
files.download(home+"/archive.zip")
